Structure of algorithms:

In [ ]:
class Algorithm:
    
    def __init__(self, estimator, setting1, setting2):
        self.estimator = estimator
        ...
        
    def run(self, problem):
        # construct circuits and observables
        # send to the estimator
        job = self.estimator.run(circuits, observables)
        ...

Question: How can I specify the transpiler steps?

High level:

In [ ]:
from qiskit_ibm_runtime import Estimator
from qiskit_ibm_runtime.options import Options

options = Options(optimization_level=1)
estimator = Estimator(options=options)

algo = Algorithm(estimator)

But:

In [ ]:
my_pm = PassManager([...])

estimator = ?

Difficulty level 2:
* Pass manager for unbound circuits: apply once, cache result
* ... and bound circuits: apply after parameter binding

In [ ]:
unbound_pm = PassManager([
    # routing, basis decomposition, gate cancellations, ...
])

bound_pm = PassManager([
    # decompose RZX & co to efficient pulse sequence
])

### Old, `QuantumInstance` solution

In [ ]:
class Algorithm:
    def __init__(self, quantum_instance, setting1, setting2):
        self.quantum_instance = quantum_instance 
        ...
        
    def run(self, problem):
        self.quantum_instance.transpile(...)

In [ ]:
quantum_instance = QuantumInstance(
    pass_manager=unbound_pm,
    bound_pass_manager=bound_pm
)

algo = Algorithm(quantum_instance)

### Solution 1 -- si!

Make `PassManager` serializable and attach to the primitive:

In [ ]:
options = Options(
    pass_manager=unbound_pm,
    bound_pass_manager=bound_pm
)
estimator = Estimator(options=options)

algo = Algorithm(estimator)

* compilation remains job of the primitives (allows offloading heavy tasks too)
* keep algorithm interfaces separated from compiling
* (no QI replacement)

Problem: what about custom passes? 
* Not safely serializable
* Can they maybe run locally 
    * What about preprocessing? Does everything happen locally up until the point where we can offload to the server? --> Yes!

### Solution 2 -- no nos gusta!

Attach the pass managers to the algorithm and skip transpilation on the primitives.

In [ ]:
class EveryAlgorithm:
    def __init__(self, estimator, setting1, setting2, unbound_pm=None, bound_pm=None):
        self.estimator = estimator
        ...
        
    def run(self, problem):
        if self.bound_pm is not None and self.unbound_pm is not None:
            skip_transpilation = True
        
        # transpile circuits
        self.estimator.run(circuit, observable, skip_transpilation=skip_transpilation)

* Couples algorithms and compiling
* Need to do this for all algorithms --> wrapping primitives convenient --> new "QI"?

In [10]:
import numpy as np
from qiskit.tools.events import TextProgressBar

iterations = 100
t = TextProgressBar()
t.start(iterations=iterations)

for i in range(iterations):
    # step i of heavy calculation ...
    t.update(i + 1)  # update progress bar

|██████████████████████████████████████████████████| 100/100 [00:00:00:00]


In [5]:
t.update(10)

|█████---------------------------------------------| 10/100 [00:00:03:28]

In [3]:
%qiskit_progress_bar -t text
_ = [np.sin(x) for x in np.linspace(0,10,100)];